In [468]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

In [469]:
# Объявление констант
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = "/data/chromedriver" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir = "data" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" #Ваш user-agent. Узнать можно тут https://юзерагент.рф, смотреть с браузера Chrome
start_time = time() # время начала выполнения программы

In [470]:
def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [471]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", 
                          #"title", 
                          "url"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [472]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [473]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []

    '''
    # ищем в объекте commenst
    td_blocks = soup.find_all("span", class_="subtext")
    print(soup.find_all("href", class_="subtext"))
    for td in td_blocks:
        comments = td.find_all("a")[3]["href"]
        print(comments)
        print(soup.find(href=comments))
    '''

    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]
        
# иногда статья располагается не на внешнем сайте, а на ycombinator, тогда article_url у нее не полный, а добавочный, с параметрами. Например item?id=200933. Для этих случаев будем добавлять урл до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
	# иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"      
        
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            #"title": tr.find(class_="storylink").string,
            "url": article_url,
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
            
    return output_list

In [474]:
# инициализируем веб драйвер
browser = webdriver.Chrome(
    #executable_path=driver_path
)

# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")
    
# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

getting page 0...
[]
HTTPSConnectionPool(host='www.bbc.com', port=443): Read timed out. (read timeout=3.0)
getting page 1...
[]
HTTPSConnectionPool(host='www.bbc.com', port=443): Read timed out. (read timeout=3.0)
getting page 2...


KeyboardInterrupt: 

In [ ]:
#hnmain > tbody > tr:nth-child(3) > td > table > tbody > tr:nth-child(2) > td.subtext > span > a:nth-child(6)
#//*[@id="hnmain"]/tbody/tr[3]/td/table/tbody/tr[2]/td[2]/span/a[3]

SyntaxError: invalid syntax (338975100.py, line 2)